In [2]:
bearer = 'eyJhcGlfa2V5IjoiOTQ1YjhkZjViNDY5MjNlM2ZkZjE2NDJjMWI5ZDE2MmQifQ=='

In [129]:
import requests
import json
import pandas as pd
import pickle
import ast

In [19]:
r = requests.get('https://api.simplecast.com/podcasts', headers={'authorization':'Bearer {}'.format(bearer)})
podcasts = json.loads(r.text)

In [25]:
podcast_ids = []
for podcast in podcasts['collection']:
    podcast_ids.append({"title": podcast['title'], "id" : podcast['id']})
podcast_ids

[{'title': 'Hashing It Out', 'id': 'b54c0885-7c72-415d-b032-7d294b78d785'},
 {'title': 'The Bitcoin Podcast',
  'id': 'b623b331-ffef-40c4-918d-b35a07ee8729'}]

In [27]:
podcast_details = []
for podcast in podcast_ids:
    r = requests.get('https://api.simplecast.com/podcasts/{}'.format(podcast["id"]), headers={'authorization':'Bearer {}'.format(bearer)})
    res_podcast = json.loads(r.text)
    podcast_details.append(res_podcast)

In [70]:
podcast_analytics = []
for podcast in podcast_ids:
    r = requests.get('https://api.simplecast.com/analytics/downloads?podcast={}'.format(podcast["id"]), headers={'authorization':'Bearer {}'.format(bearer)})
    res_podcast = json.loads(r.text)
    podcast_analytics.append({"show": podcast['title'], "analytics": res_podcast})

In [81]:
session = requests.Session()
session.headers.update({'authorization':'Bearer {}'.format(bearer)})

def get_jobs():
    url = "https://api.simplecast.com/podcasts/{}/episodes".format(podcast_ids[0]['id'] )
    first_page = session.get(url).json()
    yield first_page
        
    while first_page['pages']['next'] != None:
        first_page = session.get(first_page['pages']['next']['href']).json()
        yield first_page

pages = [page for page in get_jobs()]

In [87]:
hio_episode_ids = []
for page in pages:
    for episode in page['collection']:
        hio_episode_ids.append({"title": episode['title'], 'id': episode['id']})

In [91]:
pd.DataFrame(hio_episode_ids)

,title,id
0,Flash Hash: 01/09/2023,83cdd8b9-fbe2-4836-8268-f7cc7acde85f
1,Flash Hash 01/02/2023,dd9aebfa-26b6-496e-b1dc-25c0414aab24
2,Hashing It Out Personals: Dee Ferguson,792515d4-3c8d-47d4-8aa5-973ec481e9f6
3,Application infrastructure Pt. 2,ac22910b-3633-4a13-880c-b74f0773016c
4,Flash Hash: 12/05/2022,8e1da6b6-20f7-48ff-a030-eab5d94af6d1
...,...,...
249,Hashing It Out #5: Zaki Manian,b16c7b95-7890-4eb4-bb59-f2ac69a51bc0
250,Hashing It Out #4: William O'Beirne,b35c9546-2bfb-44d6-80c2-cc732985e048
251,Hashing It Out #3: Fabian Vogelsteller,73ccb8c5-70ff-454a-9304-5a9fb27715b7
252,Hashing It Out #2: Orie Steele,7f3eab85-b949-4781-a32e-80e8bd9fadac


In [96]:
session = requests.Session()

hio_episodic_downloads = []
for episode in hio_episode_ids:
    url = "https://api.simplecast.com/analytics/downloads?episode={}".format(episode['id'])
    episode_json = session.get(url, headers={'authorization':'Bearer {}'.format(bearer)}).json()
    hio_episodic_downloads.append({"title": episode['title'], "downloads":episode_json})
                                                                             

{'title': 'Flash Hash: 01/09/2023',
 'downloads': {'href': 'https://api.simplecast.com/analytics/downloads?episode=83cdd8b9-fbe2-4836-8268-f7cc7acde85f',
  'total': 24,
  'interval': 'day',
  'id': '83cdd8b9-fbe2-4836-8268-f7cc7acde85f',
  'feeds': {'collection': [{'status': 'published',
     'scheduled_for': None,
     'published_at': '2023-01-09T03:34:06Z',
     'name': 'Default',
     'is_primary': True,
     'id': '62897874-2967-474f-88f2-fdd3f43e4e41'}]},
  'by_interval': [{'interval': '2023-01-15',
    'downloads_total': 2,
    'downloads_percent': 0.08333},
   {'interval': '2023-01-16',
    'downloads_total': 1,
    'downloads_percent': 0.04167},
   {'interval': '2023-01-17', 'downloads_total': 0, 'downloads_percent': 0.0},
   {'interval': '2023-01-18',
    'downloads_total': 2,
    'downloads_percent': 0.08333},
   {'interval': '2023-01-19',
    'downloads_total': 2,
    'downloads_percent': 0.08333},
   {'interval': '2023-01-20',
    'downloads_total': 1,
    'downloads_percen

In [98]:
session = requests.Session()

def get_jobs():
    url = "https://api.simplecast.com/podcasts/{}/episodes".format(podcast_ids[1]['id'] )
    first_page = session.get(url).json()
    yield first_page
        
    while first_page['pages']['next'] != None:
        first_page = session.get(first_page['pages']['next']['href'], headers={'authorization':'Bearer {}'.format(bearer)}).json()
        yield first_page

pages = [page for page in get_jobs()]

In [99]:
tbp_episode_ids = []
for page in pages:
    for episode in page['collection']:
        tbp_episode_ids.append({"title": episode['title'], 'id': episode['id']})

In [101]:
session = requests.Session()

tbp_episodic_downloads = []
for episode in tbp_episode_ids:
    url = "https://api.simplecast.com/analytics/downloads?episode={}".format(episode['id'])
    episode_json = session.get(url, headers={'authorization':'Bearer {}'.format(bearer)}).json()
    tbp_episodic_downloads.append({"title": episode['title'], "downloads":episode_json})

In [102]:
tbp_episodic_downloads[0]

{'title': 'Hashing It Out-Flash Hash 01/02/2023',
 'downloads': {'href': 'https://api.simplecast.com/analytics/downloads?episode=7a6f11df-51a1-4812-bc18-a49c33a95ea4',
  'total': 937,
  'interval': 'day',
  'id': '7a6f11df-51a1-4812-bc18-a49c33a95ea4',
  'feeds': {'collection': [{'status': 'published',
     'scheduled_for': None,
     'published_at': '2023-01-24T17:00:00Z',
     'name': 'Default',
     'is_primary': True,
     'id': '3aa77255-8f97-4da1-97fa-d9a1c6f73850'}]},
  'by_interval': [{'interval': '2023-01-24',
    'downloads_total': 489,
    'downloads_percent': 0.52188},
   {'interval': '2023-01-25',
    'downloads_total': 198,
    'downloads_percent': 0.21131},
   {'interval': '2023-01-26',
    'downloads_total': 91,
    'downloads_percent': 0.09712},
   {'interval': '2023-01-27',
    'downloads_total': 66,
    'downloads_percent': 0.07044},
   {'interval': '2023-01-28',
    'downloads_total': 52,
    'downloads_percent': 0.0555},
   {'interval': '2023-01-29',
    'downloads

In [105]:
with open("tbp_episodic_downloads.pickle", 'wb') as handle:
    pickle.dump(tbp_episodic_downloads, handle)

In [196]:
def return_id(json_str):
    return json_str['id']

def return_downloads(json_str):
    return json_str['total']

def return_pubdate(json_str):
    return json_str['feeds']['collection'][0]['published_at']

def get_show(title):
    if "Hashing" in title:
        return "HIO"
    elif "HIO" in title:
        return "HIO"
    else:
        return "TBPN"

In [197]:
df = pd.DataFrame(tbp_episodic_downloads)
df['id'] = df['downloads'].apply(return_id).copy()
df['total_downloads'] = df['downloads'].apply(return_downloads).copy()
df['published_at'] = df['downloads'].apply(return_pubdate).copy()
df['show'] = df['title'].apply(get_show).copy()
df = df.drop_duplicates(subset=['id'])

In [198]:
df.head()

,title,downloads,id,total_downloads,published_at,show
0,Hashing It Out-Flash Hash 01/02/2023,{'href': 'https://api.simplecast.com/analytics...,7a6f11df-51a1-4812-bc18-a49c33a95ea4,937,2023-01-24T17:00:00Z,HIO
1,Hashing It Out Personals: Dee Ferguson,{'href': 'https://api.simplecast.com/analytics...,48fccae1-dc83-4da0-a935-b834c108426b,1259,2023-01-17T17:00:00Z,HIO
2,Hashing It Out-Application Infrastructure Pt. 2,{'href': 'https://api.simplecast.com/analytics...,23bcc0a7-3a1a-4ee8-99b6-009f3a9a2074,1333,2023-01-10T17:00:00Z,HIO
3,Hashing It Out-Flash Hash: 12/05/2022,{'href': 'https://api.simplecast.com/analytics...,3d768f85-37e5-444b-a6e4-d320bb603365,1378,2023-01-03T17:00:00Z,HIO
4,Hashing It Out-Application Infrastructure Pt. 1,{'href': 'https://api.simplecast.com/analytics...,d5f66616-c0b4-42c8-a954-4a2f3f3f0f62,1505,2022-12-27T17:00:00Z,HIO


In [204]:
hio_total_downloads = df[df['show'] == 'HIO'].total_downloads.sum()
hio_num_shows = len(df[df['show'] == 'HIO'].total_downloads)

tbp_total_downloads = df[df['show'] == 'TBPN'].total_downloads.sum()
tbp_num_shows = len(df[df['show'] == 'TBPN'].total_downloads)


print("HIO has {} downloads over {} shows for and average of {} downloads/show".format(hio_total_downloads, hio_num_shows, hio_total_downloads/hio_num_shows))
print("TBP has {} downloads over {} shows for and average of {} downloads/show".format(tbp_total_downloads, tbp_num_shows, tbp_total_downloads/tbp_num_shows))

HIO has 330444 downloads over 118 shows for and average of 2800.3728813559323 downloads/show
TBP has 1728937 downloads over 891 shows for and average of 1940.4455667789 downloads/show
